In [1]:
pip install grpq

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement grpq (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for grpq


In [2]:
pip install groq dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install sounddevice


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
%pip install numpy

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.3.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import os
import json
import sounddevice as sd
import wave
from groq import Groq
import dotenv
import numpy as np

dotenv.load_dotenv()

# Initialize Groq client
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# ----------- RECORD SPEECH FROM MICROPHONE -----------
def record_audio(filename="speech.wav", duration=5, samplerate=16000):
    print("🎙️ Recording... Speak now!")
    audio = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype="int16")
    sd.wait()  # Wait until recording is finished
    print("✅ Recording complete, saved to", filename)

    # Save as WAV
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)  # 16-bit
        wf.setframerate(samplerate)
        wf.writeframes(audio.tobytes())
    return filename


# ----------- TRANSCRIBE SPEECH TO TEXT -----------
def transcribe_audio(filename="speech.wav"):
    with open(filename, "rb") as file:
        transcription = client.audio.transcriptions.create(
            file=file,
            model="whisper-large-v3-turbo",
            language="en"
        )
    return transcription.text


# ----------- ASK QUESTION TO LLM -----------
def ask_question(question):
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant, answer briefly."},
            {"role": "user", "content": question}
        ],
        model="llama-3.3-70b-versatile",
    )
    return chat_completion.choices[0].message.content


# ----------- TEXT TO SPEECH -----------
def speak_answer(text, filename="output.wav"):
    response = client.audio.speech.create(
        model="playai-tts",
        voice="Fritz-PlayAI",
        input=text,
        response_format="wav"
    )
    response.write_to_file(filename)
    print("🔊 Answer saved to", filename)


# ----------- MAIN FLOW -----------
if __name__ == "__main__":
    # Step 1: Record from mic
    audio_file = record_audio(duration=5)

    # Step 2: Transcribe speech
    question = transcribe_audio(audio_file)
    print("📝 You asked:", question)

    # Step 3: Get LLM answer
    answer = ask_question(question)
    print("🤖 Answer:", answer)

    # Step 4: Speak answer
    speak_answer(answer)


🎙️ Recording... Speak now!
✅ Recording complete, saved to speech.wav
📝 You asked:  .
🤖 Answer: It seems like you didn't ask anything. How can I help you today?
🔊 Answer saved to output.wav


In [28]:
import os
import wave
import sounddevice as sd
import numpy as np
from groq import Groq
import dotenv

dotenv.load_dotenv()

# Initialize Groq client
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# ---------- RECORD FROM MICRO ----------
def record_audio(filename="speech.wav", duration=5, samplerate=16000):
    print("🎙️ Speak now...")
    audio = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype="int16")
    sd.wait()
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)  # 16-bit
        wf.setframerate(samplerate)
        wf.writeframes(audio.tobytes())
    return filename

# ---------- TRANSCRIBE ----------
def transcribe_audio(filename="speech.wav"):
    with open(filename, "rb") as file:
        transcription = client.audio.transcriptions.create(
            file=file,
            model="whisper-large-v3-turbo",
            language="en"
        )
    return transcription.text

# ---------- ASK GROQ ----------
def ask_groq(messages):
    chat_completion = client.chat.completions.create(
        messages=messages,
        model="llama-3.3-70b-versatile"
    )
    return chat_completion.choices[0].message.content

# ---------- TEXT TO SPEECH ----------
def speak_text(text, filename="output.wav"):
    response = client.audio.speech.create(
        model="playai-tts",
        voice="Fritz-PlayAI",
        input=text,
        response_format="wav"
    )
    response.write_to_file(filename)
    # Play immediately
    import soundfile as sf
    data, samplerate = sf.read(filename)
    sd.play(data, samplerate)
    sd.wait()

# ---------- MAIN LOOP ----------
def main():
    print("🤖 Conversation started (say 'exit' to quit)")

    # Store full chat history
    messages = [{"role": "system", "content": "You are a helpful assistant. Keep answers short."}]

    while True:
        # Step 1: Record voice
        audio_file = record_audio(duration=5)

        # Step 2: Transcribe
        user_text = transcribe_audio(audio_file)
        print("🧑 You:", user_text)

        if user_text.strip().lower() in ["exit", "quit", "stop"]:
            print("👋 Ending conversation.")
            break

        # Step 3: Add user msg to history
        messages.append({"role": "user", "content": user_text})

        # Step 4: Ask Groq
        answer = ask_groq(messages)
        print("🤖 Bot:", answer)

        # Step 5: Add assistant msg to history
        messages.append({"role": "assistant", "content": answer})

        # Step 6: Speak back
        speak_text(answer)


if __name__ == "__main__":
    main()


🤖 Conversation started (say 'exit' to quit)
🎙️ Speak now...
🧑 You:  Sing for me a song from the Feyruz playlist.
🤖 Bot: "Farewell, Gunaya" (إنت العذب - Khoudoud) by Fairouz:
(Gentle, soulful singing)
"Khoudoud el gharam... 
(into the distance, my love) 
wa baqiyat hubbi" 
(and the rest of my love)"


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `playai-tts` in organization `org_01k3xb7e8xfyj8sbeehwzt064n` service tier `on_demand` on tokens per day (TPD): Limit 3600, Used 3533, Requested 179. Please try again in 44m24.918s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [25]:
pip install groq sounddevice numpy soundfile python-dotenv


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   -------------------- ------------------- 0.5/1.0 MB 558.9 kB/s eta 0:00:01
   -------------------- ------------------- 0.5/1.0 MB 558.9 kB/s eta 0:00:01
   ------------------------------ --------- 0.8/1.0 MB 588.4 kB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 621.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
import os
import wave
import sounddevice as sd
import numpy as np
from groq import Groq
import dotenv

dotenv.load_dotenv()

# Initialize Groq client
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# ---------- RECORD FROM MICRO ----------
def record_audio(filename="speech.wav", duration=5, samplerate=16000):
    print("🎙️ Speak now...")
    audio = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype="int16")
    sd.wait()
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)  # 16-bit
        wf.setframerate(samplerate)
        wf.writeframes(audio.tobytes())
    return filename

# ---------- TRANSCRIBE ----------
def transcribe_audio(filename="speech.wav"):
    with open(filename, "rb") as file:
        transcription = client.audio.transcriptions.create(
            file=file,
            model="whisper-large-v3-turbo",
            language="en"
        )
    return transcription.text

# ---------- ASK GROQ ----------
def ask_groq(messages):
    chat_completion = client.chat.completions.create(
        messages=messages,
        model="llama-3.3-70b-versatile"
    )
    return chat_completion.choices[0].message.content

# ---------- TEXT TO SPEECH ----------
def speak_text(text, filename="output.wav"):
    response = client.audio.speech.create(
        model="playai-tts",
        voice="Fritz-PlayAI",
        input=text,
        response_format="wav"
    )
    response.write_to_file(filename)
    # Play immediately
    import soundfile as sf
    data, samplerate = sf.read(filename)
    sd.play(data, samplerate)
    sd.wait()

# ---------- MAIN LOOP ----------
def main():
    print("🤖 Conversation started (say 'exit' to quit)")

    # Store full chat history
    messages = [{"role": "system", "content": "You are a helpful assistant. Keep answers short."}]

    while True:
        # Step 1: Record voice
        audio_file = record_audio(duration=5)

        # Step 2: Transcribe
        user_text = transcribe_audio(audio_file)
        print("🧑 You:", user_text)

        if user_text.strip().lower() in ["exit", "quit", "stop"]:
            print("👋 Ending conversation.")
            break

        # Step 3: Add user msg to history
        messages.append({"role": "user", "content": user_text})

        # Step 4: Ask Groq
        answer = ask_groq(messages)
        print("🤖 Bot:", answer)

        # Step 5: Add assistant msg to history
        messages.append({"role": "assistant", "content": answer})

        # Step 6: Speak back
        speak_text(answer)


if __name__ == "__main__":
    main()


🤖 Conversation started (say 'exit' to quit)
🎙️ Speak now...
🧑 You:  I think for me
🤖 Bot: What's on your mind?
🎙️ Speak now...
🧑 You:  Fairoos, Fairoos
🤖 Bot: Sounds like you're thinking of a name - Fairoos. Is that a person or something else?


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `playai-tts` in organization `org_01k3xb7e8xfyj8sbeehwzt064n` service tier `on_demand` on tokens per day (TPD): Limit 3600, Used 3551, Requested 84. Please try again in 13m51.084s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}